# Differential Drive Forward Kinematics — Interactive (Binder-ready)

This notebook teaches the fundamentals of **differential drive** forward kinematics with interactive demos.
You will:
- derive body-frame velocities from wheel speeds
- integrate pose $(x, y, \theta)$ forward in time
- visualize trajectories and special cases (straight, in-place rotation, arcs)
- (optional) compare **Euler** vs **exact** integration for constant controls

---


## Environment sanity check

Run this cell first. You should see a slider widget and a simple plot.


In [ ]:
import importlib

required = ["numpy", "matplotlib", "ipywidgets"]
missing = [p for p in required if importlib.util.find_spec(p) is None]
for p in required:
    print(("✓ " if p not in missing else "✗ ") + p)
if missing:
    raise ImportError(f"Missing required packages: {missing}")

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display

display(widgets.IntSlider(description="Widget test"))

plt.figure(figsize=(4,3))
plt.plot([0,1],[0,1])
plt.title("Matplotlib sanity check")
plt.grid(True, alpha=0.3)
plt.show()

print("Sanity check complete.")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display


# 1) Robot model and notation

A differential drive robot has two wheels separated by track width (axle length) $L$ and wheel radius $r$.

Let wheel angular velocities be:
- $\omega_R$ (right wheel), $\omega_L$ (left wheel) in rad/s.

Wheel **linear** speeds:
$$
v_R = r\,\omega_R,\qquad v_L = r\,\omega_L.
$$

Body-frame forward speed (at the midpoint between wheels) and yaw rate:
$$
v = \frac{v_R + v_L}{2} = \frac{r}{2}(\omega_R + \omega_L)
$$
$$
\dot{\theta} = \omega = \frac{v_R - v_L}{L} = \frac{r}{L}(\omega_R - \omega_L).
$$

The pose is $\mathbf{x} = (x, y, \theta)$ in the world frame. The continuous-time kinematics are:
$$
\dot{x} = v\cos\theta,\qquad \dot{y} = v\sin\theta,\qquad \dot{\theta} = \omega.
$$

---


# 2) From wheel speeds to $(v,\omega)$ (interactive)

Move the sliders and observe how $v$ and $\omega$ change.

Special cases:
- $\omega_R = \omega_L$ → straight line ($\omega=0$)
- $\omega_R = -\omega_L$ → in-place rotation ($v=0$)
- one wheel faster → arc motion


In [ ]:
def vw_from_wheels(omega_L, omega_R, r, L):
    vL = r * omega_L
    vR = r * omega_R
    v = 0.5 * (vR + vL)
    w = (vR - vL) / L
    return v, w, vL, vR

omegaL = widgets.FloatSlider(value=2.0, min=-10, max=10, step=0.1, description="ω_L (rad/s)")
omegaR = widgets.FloatSlider(value=4.0, min=-10, max=10, step=0.1, description="ω_R (rad/s)")
r = widgets.FloatSlider(value=0.05, min=0.01, max=0.20, step=0.005, description="r (m)")
L = widgets.FloatSlider(value=0.30, min=0.10, max=0.80, step=0.01, description="L (m)")
out = widgets.Output()

def update_vw(omega_L, omega_R, r, L):
    v, w, vL, vR = vw_from_wheels(omega_L, omega_R, r, L)
    with out:
        out.clear_output()
        print(f"v_L = {vL:.3f} m/s   v_R = {vR:.3f} m/s")
        print(f"v   = {v:.3f} m/s")
        print(f"ω   = {w:.3f} rad/s  ({np.rad2deg(w):.2f} deg/s)")
        if abs(w) < 1e-8:
            print("Motion type: straight line (ω≈0)")
        elif abs(v) < 1e-8:
            print("Motion type: in-place rotation (v≈0)")
        else:
            R = v / w
            print(f"Motion type: circular arc with radius R=v/ω={R:.3f} m")

ui = widgets.VBox([widgets.HBox([omegaL, omegaR]), widgets.HBox([r, L])])
widgets.interactive_output(update_vw, {"omega_L": omegaL, "omega_R": omegaR, "r": r, "L": L})
display(ui, out)
update_vw(omegaL.value, omegaR.value, r.value, L.value)


# 3) Discrete-time forward integration

To predict the next pose, integrate over timestep $\Delta t$.

## Euler (forward) integration
$$
x_{k+1} = x_k + \Delta t\, v\cos\theta_k
$$
$$
y_{k+1} = y_k + \Delta t\, v\sin\theta_k
$$
$$
\theta_{k+1} = \theta_k + \Delta t\, \omega
$$

## Exact integration for constant $(v,\omega)$ over $\Delta t$
If $\omega \neq 0$:
$$
x_{k+1} = x_k + \frac{v}{\omega}\Big(\sin(\theta_k+\omega\Delta t) - \sin\theta_k\Big)
$$
$$
y_{k+1} = y_k - \frac{v}{\omega}\Big(\cos(\theta_k+\omega\Delta t) - \cos\theta_k\Big)
$$
$$
\theta_{k+1} = \theta_k + \omega\Delta t
$$

If $\omega = 0$, it reduces to straight-line motion.

Below, compare Euler vs exact trajectories.


In [ ]:
def wrap_angle(theta):
    return (theta + np.pi) % (2*np.pi) - np.pi

def step_euler(x, y, th, v, w, dt):
    return (x + dt * v * np.cos(th),
            y + dt * v * np.sin(th),
            wrap_angle(th + dt * w))

def step_exact(x, y, th, v, w, dt):
    if abs(w) < 1e-10:
        return step_euler(x, y, th, v, w, dt)
    th2 = th + w*dt
    x2 = x + (v/w) * (np.sin(th2) - np.sin(th))
    y2 = y - (v/w) * (np.cos(th2) - np.cos(th))
    return x2, y2, wrap_angle(th2)

def rollout(v, w, dt, steps, method="exact", x0=0.0, y0=0.0, th0=0.0):
    xs = np.zeros(steps+1); ys = np.zeros(steps+1); ths = np.zeros(steps+1)
    xs[0], ys[0], ths[0] = x0, y0, th0
    fn = step_exact if method == "exact" else step_euler
    x, y, th = x0, y0, th0
    for k in range(steps):
        x, y, th = fn(x, y, th, v, w, dt)
        xs[k+1], ys[k+1], ths[k+1] = x, y, th
    return xs, ys, ths

def draw_robot(ax, x, y, th, L=0.3):
    fwd = np.array([np.cos(th), np.sin(th)])
    left = np.array([-np.sin(th), np.cos(th)])
    p1 = np.array([x, y]) + 0.18*L*fwd
    p2 = np.array([x, y]) - 0.12*L*fwd + 0.10*L*left
    p3 = np.array([x, y]) - 0.12*L*fwd - 0.10*L*left
    ax.fill([p1[0], p2[0], p3[0]], [p1[1], p2[1], p3[1]], alpha=0.4)

def plot_rollouts(xs_exact, ys_exact, xs_euler=None, ys_euler=None, title="Trajectory"):
    fig, ax = plt.subplots(figsize=(6.5, 6.5))
    ax.grid(True, alpha=0.3)
    ax.set_aspect("equal", adjustable="box")
    ax.plot(xs_exact, ys_exact, label="exact")
    ax.scatter(xs_exact[0], ys_exact[0], s=50, marker="o")
    ax.scatter(xs_exact[-1], ys_exact[-1], s=70, marker="x")
    if xs_euler is not None:
        ax.plot(xs_euler, ys_euler, linestyle="--", label="euler")
        ax.scatter(xs_euler[-1], ys_euler[-1], s=70, marker="x")
        ax.legend()
    ax.set_xlabel("x (m)")
    ax.set_ylabel("y (m)")
    ax.set_title(title)
    return fig, ax


In [ ]:
omegaL2 = widgets.FloatSlider(value=2.0, min=-10, max=10, step=0.1, description="ω_L")
omegaR2 = widgets.FloatSlider(value=4.0, min=-10, max=10, step=0.1, description="ω_R")
r2 = widgets.FloatSlider(value=0.05, min=0.01, max=0.20, step=0.005, description="r (m)")
L2 = widgets.FloatSlider(value=0.30, min=0.10, max=0.80, step=0.01, description="L (m)")
dt = widgets.FloatSlider(value=0.05, min=0.01, max=0.30, step=0.01, description="Δt (s)")
steps = widgets.IntSlider(value=150, min=10, max=400, step=10, description="steps")
theta0 = widgets.FloatSlider(value=0.0, min=-180, max=180, step=1, description="θ0 (deg)")
compare = widgets.Checkbox(value=True, description="Compare Euler vs exact")

out_traj = widgets.Output()

def update_traj(omega_L, omega_R, r, L, dt, steps, theta0_deg, compare=True):
    v, w, *_ = vw_from_wheels(omega_L, omega_R, r, L)
    th0 = np.deg2rad(theta0_deg)
    xsE, ysE, thsE = rollout(v, w, dt, steps, method="exact", th0=th0)
    xsU = ysU = None
    if compare:
        xsU, ysU, _ = rollout(v, w, dt, steps, method="euler", th0=th0)

    with out_traj:
        out_traj.clear_output()
        title = f"ω_L={omega_L:.1f}, ω_R={omega_R:.1f}, r={r:.3f}, L={L:.3f} → v={v:.3f} m/s, ω={w:.3f} rad/s"
        fig, ax = plot_rollouts(xsE, ysE, xsU, ysU, title=title)
        draw_robot(ax, xsE[-1], ysE[-1], thsE[-1], L=L)
        plt.show()

ui = widgets.VBox([
    widgets.HBox([omegaL2, omegaR2]),
    widgets.HBox([r2, L2]),
    widgets.HBox([dt, steps]),
    widgets.HBox([theta0, compare])
])
widgets.interactive_output(update_traj, {
    "omega_L": omegaL2, "omega_R": omegaR2, "r": r2, "L": L2,
    "dt": dt, "steps": steps, "theta0_deg": theta0, "compare": compare
})
display(ui, out_traj)
update_traj(omegaL2.value, omegaR2.value, r2.value, L2.value, dt.value, steps.value, theta0.value, compare.value)


# 4) Instantaneous Center of Curvature (ICC)

When $\omega \neq 0$, the robot follows a circular arc of radius:
$$
R = \frac{v}{\omega}
$$

The ICC is located at:
$$
\text{ICC} =
\begin{bmatrix} x \\ y \end{bmatrix}
+ R\begin{bmatrix} -\sin\theta \\ \cos\theta \end{bmatrix}.
$$

Below, visualize the ICC and the instantaneous turning circle for the current pose.


In [ ]:
x0 = widgets.FloatSlider(value=0.0, min=-2, max=2, step=0.05, description="x0")
y0 = widgets.FloatSlider(value=0.0, min=-2, max=2, step=0.05, description="y0")
th0 = widgets.FloatSlider(value=0.0, min=-180, max=180, step=1, description="θ0 (deg)")
omegaL3 = widgets.FloatSlider(value=2.0, min=-10, max=10, step=0.1, description="ω_L")
omegaR3 = widgets.FloatSlider(value=4.0, min=-10, max=10, step=0.1, description="ω_R")
r3 = widgets.FloatSlider(value=0.05, min=0.01, max=0.20, step=0.005, description="r")
L3 = widgets.FloatSlider(value=0.30, min=0.10, max=0.80, step=0.01, description="L")

out_icc = widgets.Output()

def icc_demo(x0, y0, th0_deg, omega_L, omega_R, r, L):
    th = np.deg2rad(th0_deg)
    v, w, *_ = vw_from_wheels(omega_L, omega_R, r, L)

    fig, ax = plt.subplots(figsize=(6.5, 6.5))
    ax.grid(True, alpha=0.3)
    ax.set_aspect("equal", adjustable="box")
    ax.set_xlim(-3, 3)
    ax.set_ylim(-3, 3)
    ax.set_xlabel("x (m)")
    ax.set_ylabel("y (m)")
    ax.set_title("Instantaneous Center of Curvature (ICC)")

    ax.scatter([x0], [y0], s=60)
    draw_robot(ax, x0, y0, th, L=L)
    ax.plot([x0, x0 + 0.6*np.cos(th)], [y0, y0 + 0.6*np.sin(th)], linewidth=2, alpha=0.6)

    if abs(w) < 1e-8:
        ax.text(-2.8, 2.6, "ω≈0 → straight line (ICC at infinity)",
                bbox=dict(boxstyle="round", alpha=0.15))
    else:
        R = v / w
        icc = np.array([x0, y0]) + R * np.array([-np.sin(th), np.cos(th)])
        ax.scatter([icc[0]], [icc[1]], s=90, marker="x")
        ax.text(icc[0]+0.05, icc[1]+0.05, "ICC")

        ang = np.linspace(0, 2*np.pi, 250)
        ax.plot(icc[0] + abs(R)*np.cos(ang), icc[1] + abs(R)*np.sin(ang), linestyle="--", alpha=0.6)
        ax.text(-2.8, 2.3, f"v={v:.3f} m/s, ω={w:.3f} rad/s
R=v/ω={R:.3f} m",
                bbox=dict(boxstyle="round", alpha=0.15))

    plt.show()

ui = widgets.VBox([
    widgets.HBox([x0, y0, th0]),
    widgets.HBox([omegaL3, omegaR3]),
    widgets.HBox([r3, L3])
])
widgets.interactive_output(icc_demo, {
    "x0": x0, "y0": y0, "th0_deg": th0,
    "omega_L": omegaL3, "omega_R": omegaR3,
    "r": r3, "L": L3
})
display(ui, out_icc)
icc_demo(x0.value, y0.value, th0.value, omegaL3.value, omegaR3.value, r3.value, L3.value)


# 5) Quick checks for students

1. Set $\omega_L=\omega_R=5$ rad/s. What is $\omega$? What does the trajectory look like?
2. Set $\omega_L=-\omega_R$. What is $v$? What does the robot do?
3. Fix $\omega_R$ and slowly increase $\omega_L$. How does turning radius change?

---


# 6) Going further

Possible follow-ups:
- odometry from encoder ticks (dead reckoning)
- noise and drift, and how to mitigate it
- inverse kinematics: desired $(v,\omega)$ → wheel speeds
- tracking control (e.g., pure pursuit) using this model
